In [1]:
# Imported required libraries for API requests and data handling
# =============================================================================
# requests: For making HTTP requests to the CoinGecko API
# os: For accessing environment variables
# dotenv: For loading API key from .env file securely
# pandas: For data manipulation and DataFrame operations

import requests as rq
import os
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Fetched Bitcoin price data from CoinGecko API for specified date range

# Defined date range for Bitcoin price data (August to December 2025)
begin_date = "2025-08-01" 
end_date = "2025-12-31"

# Built API endpoint URL for historical Bitcoin price data
# vs_currency=usd returns prices in US dollars
url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from={begin_date}&to={end_date}"

# Loaded API key from .env file for authentication
load_dotenv()
headers = {"x-cg-api-key": os.getenv('API_KEY') }

# Made GET request to CoinGecko API
response = rq.get(url, headers=headers)

# Displayed raw JSON response to verify successful data retrieval
print(response.text)

{"prices":[[1754006400000,115700.00243915782],[1754092800000,113234.6051343189],[1754179200000,112554.90232221723],[1754265600000,114199.10966460757],[1754352000000,115138.68613070177],[1754438400000,114128.35408881678],[1754524800000,115022.09576830892],[1754611200000,117463.47451085235],[1754697600000,116688.36663186055],[1754784000000,116510.08393213755],[1754870400000,119266.92516880555],[1754956800000,118773.79960860992],[1755043200000,120202.53485503166],[1755129600000,123560.99363577305],[1755216000000,118405.59579823953],[1755302400000,117339.79190213277],[1755388800000,117501.21653394958],[1755475200000,117542.83687778088],[1755561600000,116256.41276740946],[1755648000000,112778.34483555844],[1755734400000,114252.39755195397],[1755820800000,112414.39987336512],[1755907200000,116834.24948202295],[1755993600000,115359.98346714744],[1756080000000,113399.54847314971],[1756166400000,110185.35443900425],[1756252800000,111842.70999260596],[1756339200000,111216.08479629169],[175642560

In [3]:
# Parsed JSON response and extracted price data

# Converted JSON string response into Python dictionary
data = response.json()

# Extracted only price data (ignoring market_caps and total_volumes arrays)
prices = data['prices']

# Defined column names for DataFrame creation
header_row = ['timestamp', 'price']

In [4]:
# Created DataFrame from extracted price data

# Converted list of [timestamp, price] pairs into structured DataFrame
df_2 = pd.DataFrame(prices, columns=header_row)
df_2

,timestamp,price
0,1754006400000,115700.002439
1,1754092800000,113234.605134
2,1754179200000,112554.902322
3,1754265600000,114199.109665
4,1754352000000,115138.686131
...,...,...
148,1766793600000,87305.956560
149,1766880000000,87807.003211
150,1766966400000,87822.908291
151,1767052800000,87156.562661


In [5]:
# Transformed timestamp column into separate date and time columns

# Converted Unix timestamp (in milliseconds) to readable datetime objects
# unit='ms' specifies that timestamp is in milliseconds
df_2['timestamp'] = pd.to_datetime(df_2['timestamp'], unit='ms')

# Split timestamp into separate date and time columns for flexibility
df_2['date'] = df_2['timestamp'].dt.date
df_2['time'] = df_2['timestamp'].dt.time

# Dropped the original timestamp column (no longer needed)
df_2 = df_2.drop('timestamp', axis=1)

# Reordered columns to: date, time, price
df_2 = df_2[['date', 'time', 'price']]
df_2

,date,time,price
0,2025-08-01,00:00:00,115700.002439
1,2025-08-02,00:00:00,113234.605134
2,2025-08-03,00:00:00,112554.902322
3,2025-08-04,00:00:00,114199.109665
4,2025-08-05,00:00:00,115138.686131
...,...,...,...
148,2025-12-27,00:00:00,87305.956560
149,2025-12-28,00:00:00,87807.003211
150,2025-12-29,00:00:00,87822.908291
151,2025-12-30,00:00:00,87156.562661


In [6]:
# Loaded historical cryptocurrency data from CSV file

# Read pre-existing dataset containing top 100 cryptos with network information
df_1 = pd.read_csv('data/top_100_cryptos_with_correct_network.csv')
df_1

,symbol,date,open,high,low,close,network
0,BTCUSDT,2018-08-01,7735.670,7750.000,7430.000,7604.580,Bitcoin
1,BTCUSDT,2018-08-02,7600.080,7709.460,7455.720,7525.710,Bitcoin
2,BTCUSDT,2018-08-03,7525.710,7540.000,7282.440,7418.780,Bitcoin
3,BTCUSDT,2018-08-04,7412.270,7494.810,6926.000,7009.840,Bitcoin
4,BTCUSDT,2018-08-05,7009.840,7089.870,6882.290,7024.190,Bitcoin
...,...,...,...,...,...,...,...
211674,UMAUSDT,2025-08-01,1.242,1.251,1.162,1.202,Ethereum
211675,UMAUSDT,2025-08-02,1.202,1.216,1.147,1.172,Ethereum
211676,UMAUSDT,2025-08-03,1.175,1.210,1.158,1.207,Ethereum
211677,UMAUSDT,2025-08-04,1.206,1.247,1.201,1.242,Ethereum


In [7]:
# Calculated average daily price from OHLC data

# Computed mean of open, high, low, close prices for each row
# axis=1 calculates mean across columns (horizontally)
df_1['avg_price'] = df_1[['open', 'high', 'low', 'close']].mean(axis=1)
df_1['avg_price']

0         7630.06250
1         7572.74250
2         7441.73250
3         7210.73000
4         7001.54750
             ...    
211674       1.21425
211675       1.18425
211676       1.18750
211677       1.22400
211678       1.23150
Name: avg_price, Length: 211679, dtype: float64

In [8]:
# Selected only relevant columns for analysis

# Kept symbol, date, and calculated average price columns
df_1 = df_1[['symbol', 'date', 'avg_price']]
df_1

,symbol,date,avg_price
0,BTCUSDT,2018-08-01,7630.06250
1,BTCUSDT,2018-08-02,7572.74250
2,BTCUSDT,2018-08-03,7441.73250
3,BTCUSDT,2018-08-04,7210.73000
4,BTCUSDT,2018-08-05,7001.54750
...,...,...,...
211674,UMAUSDT,2025-08-01,1.21425
211675,UMAUSDT,2025-08-02,1.18425
211676,UMAUSDT,2025-08-03,1.18750
211677,UMAUSDT,2025-08-04,1.22400


In [9]:
# Filtered data to keep only Bitcoin (BTCUSDT) records

# Kept only rows where symbol equals 'BTCUSDT' (Bitcoin against Tether)
df_1 = df_1[df_1['symbol'] == 'BTCUSDT']
df_1

,symbol,date,avg_price
0,BTCUSDT,2018-08-01,7630.0625
1,BTCUSDT,2018-08-02,7572.7425
2,BTCUSDT,2018-08-03,7441.7325
3,BTCUSDT,2018-08-04,7210.7300
4,BTCUSDT,2018-08-05,7001.5475
...,...,...,...
2557,BTCUSDT,2025-08-01,114459.1450
2558,BTCUSDT,2025-08-02,112977.6900
2559,BTCUSDT,2025-08-03,113368.7800
2560,BTCUSDT,2025-08-04,114772.8600


In [10]:
# Cleaned and prepared both DataFrames for merging

# Dropped 'symbol' column (no longer needed after filtering)
df_1 = df_1.drop(columns=['symbol'], axis=1)

# Removed 2018 data (incomplete year in the dataset)
# .astype(str).str[:4] converts date to string and extracts year (first 4 chars)
df_1 = df_1[df_1['date'].astype(str).str[:4] != '2018']

# Dropped 'time' column from API data (only need daily averages)
df_2 = df_2.drop(columns=['time'], axis=1)

print(df_1.head())
print(df_2.head())

           date  avg_price
153  2019-01-01  3737.6325
154  2019-01-02  3821.9000
155  2019-01-03  3804.2725
156  2019-01-04  3771.6050
157  2019-01-05  3788.2600
         date          price
0  2025-08-01  115700.002439
1  2025-08-02  113234.605134
2  2025-08-03  112554.902322
3  2025-08-04  114199.109665
4  2025-08-05  115138.686131


In [11]:
# Renamed columns for consistency before merging

# Standardized price column names to 'avg_daily_price' in both DataFrames
df_2 = df_2.rename(columns={'price': 'avg_daily_price'})
df_1 = df_1.rename(columns={'avg_price': 'avg_daily_price'})

print(df_1.head())
print(df_2.head())

           date  avg_daily_price
153  2019-01-01        3737.6325
154  2019-01-02        3821.9000
155  2019-01-03        3804.2725
156  2019-01-04        3771.6050
157  2019-01-05        3788.2600
         date  avg_daily_price
0  2025-08-01    115700.002439
1  2025-08-02    113234.605134
2  2025-08-03    112554.902322
3  2025-08-04    114199.109665
4  2025-08-05    115138.686131


In [12]:
# Combined both DataFrames into a single dataset

# Concatenated historical data (df_1) with new API data (df_2)
# ignore_index=True resets index to sequential numbers
combined_df = pd.concat([df_1, df_2], ignore_index=True)

print(combined_df.head())
# Checked for missing values in the combined dataset
print(combined_df.isna().sum())
# Displayed total rows and columns
print(combined_df.shape)

         date  avg_daily_price
0  2019-01-01        3737.6325
1  2019-01-02        3821.9000
2  2019-01-03        3804.2725
3  2019-01-04        3771.6050
4  2019-01-05        3788.2600
date               0
avg_daily_price    0
dtype: int64
(2562, 2)


In [13]:
# Saved combined Bitcoin price data to CSV file

# Created 'data' directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Defined output filename
filename = "data/price_of_bitcoin_from_2018_to_2025.csv"

# Attempted to save DataFrame to CSV with error handling
# index=False prevents writing row numbers to the file
try:
    combined_df.to_csv(filename, index=False)
    print(f"Data saved to '{filename}' successfully.")
except Exception as e:
    print(f"{e} was encountered while saving data to: '{filename}'.")

Data saved to 'data/price_of_bitcoin_from_2018_to_2025.csv' successfully.
